In [1]:
from dotenv import load_dotenv, find_dotenv

# read local .env file
# find_dotenv 函数通常用于搜索和定位目录树中的 .env 文件，而 load_dotenv 函数用于将 .env 文件中的变量加载到环境中。
_ = load_dotenv(find_dotenv())
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

In [4]:
"""
对话知识图谱记忆
"""
from langchain.memory import ConversationKGMemory

memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})

In [5]:
memory.load_memory_variables({"input": 'who is sam'})
 

{'history': 'On sam: sam is a friend.'}

In [ ]:
""" output
{'history': 'On sam: sam is a friend.'}
"""

In [6]:
"""
我们还可以将历史记录作为消息列表获取（如果您正在使用聊天模型，则这非常有用)。
"""
memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})
 

In [7]:
memory.load_memory_variables({"input": 'who is sam'})


{'history': [SystemMessage(content='On sam: sam is a friend.')]}

In [ ]:
""" output
{'history': [SystemMessage(content='On sam: sam is a friend.')]}
"""

In [8]:
"""
我们还可以更模块化地从新消息中获取当前实体（将先前的消息用作上下文)。
"""
memory.get_current_entities("what's Sams favorite color?")
 

['Sam']

In [ ]:
""" output
['sam']
"""

In [9]:
"""
我们还可以更模块化地从新消息中获取知识三元组（将先前的消息用作上下文)。
"""
memory.get_knowledge_triplets("her favorite color is red")


[KnowledgeTriple(subject='sam', predicate='favorite color is', object_='red')]

In [ ]:
""" output
[KnowledgeTriple(subject='sam', predicate='favorite color is', object_='red')]
"""

In [10]:
"""
在链式使用中
"""
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
 
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.
 
Relevant Information:
 
{history}
 
Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)
conversation_with_kg = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=prompt,
    memory=ConversationKGMemory(llm=llm)
)

In [11]:
conversation_with_kg.predict(input="Hi, what's up?")
 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.
 
Relevant Information:
 

 
Conversation:
Human: Hi, what's up?
AI:

> Finished chain.


"Hello! I'm an AI designed to have conversations and provide information. How can I assist you today?"

In [ ]:
""" output
"Hello! I'm an AI designed to have conversations and provide information. How can I assist you today?"
"""

In [12]:
conversation_with_kg.predict(input="My name is James and I'm helping Will. He's an engineer.")
 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.
 
Relevant Information:
 

 
Conversation:
Human: My name is James and I'm helping Will. He's an engineer.
AI:

> Finished chain.


"Hello James! It's nice to meet you. How can I help you with Will's engineering work?"

In [ ]:
""" output
"Hello James! It's nice to meet you. How can I help you with Will's engineering work?"
"""

In [13]:
conversation_with_kg.predict(input="What do you know about Will?")
 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.
 
Relevant Information:
 
On Will: Will is an engineer.
 
Conversation:
Human: What do you know about Will?
AI:

> Finished chain.


'Will is an engineer.'

In [ ]:
""" output
'Will is an engineer.'
"""